In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score

from sklearn.compose import ColumnTransformer

In [13]:
df = pd.read_csv('titanic.csv')[['Age','Fare','SibSp','Parch','Survived']]
df.dropna(inplace= True)
df.head()

,Age,Fare,SibSp,Parch,Survived
0,34.5,7.8292,0,0,0
1,47.0,7.0000,1,0,1
2,62.0,9.6875,0,0,0
3,27.0,8.6625,0,0,0
4,22.0,12.2875,1,1,1


In [14]:
df['Family']= df['SibSp']+df['Parch']
df.head()

,Age,Fare,SibSp,Parch,Survived,Family
0,34.5,7.8292,0,0,0,0
1,47.0,7.0000,1,0,1,1
2,62.0,9.6875,0,0,0,0
3,27.0,8.6625,0,0,0,0
4,22.0,12.2875,1,1,1,2


In [15]:
df.drop(columns= ['SibSp','Parch'],inplace= True)
df.head()

,Age,Fare,Survived,Family
0,34.5,7.8292,0,0
1,47.0,7.0000,1,1
2,62.0,9.6875,0,0
3,27.0,8.6625,0,0
4,22.0,12.2875,1,2


In [16]:
X = df.drop(columns=['Survived'])
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

In [17]:
# without binarization:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
accuracy_score(y_test, y_pred)

0.4925373134328358

In [18]:
np.mean(cross_val_score(dt, X, y, cv = 10, scoring= 'accuracy'))

0.604367201426025

In [19]:
# now applying binarization:
from sklearn.preprocessing import Binarizer
ct = ColumnTransformer([
    ('bin',Binarizer(copy=False),['Family'])
],remainder='passthrough')
X_train_transform = ct.fit_transform(X_train)
X_test_transform = ct.transform(X_test)

pd.DataFrame(X_train_transform,columns=['family','Age','Fare'])




,family,Age,Fare
0,1.0,22.0,59.4000
1,0.0,23.0,7.8542
2,1.0,17.0,16.1000
3,1.0,43.0,21.0000
4,0.0,36.0,7.2500
...,...,...,...
259,0.0,57.0,13.0000
260,0.0,47.0,42.4000
261,1.0,24.0,31.5000
262,0.0,21.0,7.8958


In [20]:
dt = DecisionTreeClassifier()
dt.fit(X_train_transform,y_train)
y_pred1 = dt.predict(X_test_transform)
accuracy_score(y_test, y_pred1)

0.4925373134328358

In [21]:
dt = DecisionTreeClassifier()
X_transform = ct.fit_transform(X)
np.mean(cross_val_score(dt, X_transform, y, cv= 10, scoring= 'accuracy'))

0.6044563279857398